# Mix/Fusion of Classes=Categories
version: 1

info: 
- mix/fusion: of classes into other classes
    0. If not mola.json : Manually add dataset descriptors when importing id, e.g. COCO(see below)
- reorder_ids: It also reorders category id

author: nuno costa

In [12]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random

In [13]:
#Define root dir dependent on OS
rdir_dsets='D:/external_datasets/' #WARNING: DATASETS ROOT is OK?
rdir='D:/external_datasets/MOLA/' 
#/home/user/Data/Datasets/External Datasets/MOLA/
if str(platform.platform()).find('Linux')>-1:
    rdir_dsets=rdir_dsets.replace('D:/external_datasets/','/mnt/Data/Datasets/External Datasets/')
    rdir=rdir.replace('D:/external_datasets/MOLA/','/mnt/Data/Work/EASYRIDE/P19/NC/yolov5/JSONS/') #jsons file
print('OS: {}'.format(platform.platform()))
print('root datasets dir: {}'.format(rdir_dsets))
print('root dir: {}'.format(rdir))

OS: Linux-5.11.0-37-generic-x86_64-with-glibc2.10
root datasets dir: /mnt/Data/Datasets/External Datasets/
root dir: /mnt/Data/Work/EASYRIDE/P19/NC/yolov5/JSONS/


In [63]:
#jsonfile
injsonfile="splitann_cocotaolbo_fix_equal_reorder_cleanclass_cleanimg/train" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+'annotations/'+injsonfile+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

info 5
licenses 9
categories 1298
videos 1488
images 194943
tracks 8132
segment_info 0
annotations 943300
datasets 3


## 1. Import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [64]:
# datasets name and id
dset_l=[]
dset_l_id=[]
try:
    for d in molajson['datasets']:
        dset_l.append(d['name'])
        dset_l_id.append(d['id'])
except: #manually add for example for only COCO
    dset_l=['TAO']
    dset_l_id=[1]
print(dset_l, dset_l_id)

['COCO', 'TAO', 'LBO'] [1, 2, 3]


In [65]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    try:
        cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
    except:
        cat_l_dset.append(dset_l[0])
#print(cat_l_id)

In [66]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [67]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    try:
        ann_dset.append(an['dataset'])
    except:
        ann_dset.append(dset_l_id[0])

100%|██████████| 943300/943300 [00:00<00:00, 1269979.64it/s]


## 2. Find mixers cat_ids
mixers example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [68]:
#mixers #TODO: SORT alphabetically
mixers_l=[]
mixers_l_catid=[]
mixers_l_catdset=[]
mixer_method="all_cats"
if mixer_method=="all_cats": #Do for all category names, even with equal 
    mixers_l=cat_l
    mixers_l_catid=[[id] for id in cat_l_id]
    mixers_l_catdset=[[dset] for dset in cat_l_dset]
    

print(mixers_l[0:5])
print(mixers_l_catid[0:5])
print(mixers_l_catdset[0:5])
print(len(mixers_l))
print(len(mixers_l_catid))
print(len(mixers_l_catdset))

['person', 'bicycle', 'car', 'motorcycle', 'airplane']
[[1], [2], [3], [4], [5]]
[['COCO'], ['COCO'], ['COCO'], ['COCO'], ['COCO']]
1298
1298
1298


In [69]:
# get annotations mixers
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
mixers_l_imgid=[]
mixers_l_bbox=[]
mixers_l_dset=[]
mixers_l_ann_nr=[]
for catids in tqdm(mixers_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    l_ann_nr=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
        l_ann_nr.append([len(ann_idx)])
    mixers_l_imgid.append(l_imgid)
    mixers_l_bbox.append(l_bbox)
    mixers_l_dset.append(l_dset)
    mixers_l_ann_nr.append(l_ann_nr)

100%|██████████| 1298/1298 [00:03<00:00, 376.39it/s] 


## 3. Classes|categories to mix w/ EXCEL report

In [70]:
#INIT VARS
classtomix_l=[]
classtomix_l_catid=[]
method="" #"save_images": to save new images and create excel report for manual inspection and build classtomix_l; "": use a saved excel report, uncomment excelpath
datadir="mixers/"+injsonfile+"/" #root folder to save mixer method . #WARNING mixers/original json that was used to save images and excel
folder=mixer_method+'/' #folder to save images and exel 
showimage=False #show images
startidx=0 # start index of image to save from each dataset
imgnr=1 # total number of images to save from each dataset
imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
#paths
path=os.path.join(rdir,datadir,folder) #path to folder
if method=='save_images': assure_path_exists(path)
excelpath=path+mixer_method+"_v1.xlsx"#path+mixer_method+"_classtomix_report.xlsx"#path to excel
#fixed path - if method=""
if not method: excelpath=rdir+"mixers/cocotaolbo_fix_equal_reorder_cleanclass_cleanimg/mix_cocotaolbo_aggressive_3c.xlsx"

In [71]:
#METHODS
if method=="save_images": # save images and excel report to folder for manual edit the classtomix_l
    df=pd.DataFrame({'mixers_l': mixers_l,'mixers_l_catid': mixers_l_catid, 'mixers_l_catdset': mixers_l_catdset, 'classtomix_l': np.nan, 'classtomix_l_catid':np.nan, 'rules':np.nan })
    df.loc[0, 'rules']="To fix classes: 1) You need to fill the column classtomix_l and/or classtomix_l_catid with the information from the respective mixer columns; 2) When copy/pasting or changing, make sure the same structure maintains:  ['car', 'carrot'], [3, 52], beware of the spaces ['car', '  and always maintain the first class in the list;  3) You have 3 possibilities of filling the columns : 1-the 2 columns empty, meaning the row will not be used for classtomix; 2-only one column empty, e.g. fill the classtotix_l row with the class labels from mixers_l, then during the importing the classtomix_l_catid is filled, and vice-versa; 3-If you want to change the name of the first class in the list,e.g ['car', 'carrot'] for ['automobile', 'carrot'] you need to provide the ids to classtomix_l_catid."
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    df['number_annotations'] = np.empty((len(df), 0)).tolist()
    #save image for each mixer
    for i, mixer in enumerate(tqdm(mixers_l)): #run for each mixer category
        firstclass=mixer
        if isinstance(firstclass, list): firstclass=firstclass[0] #first class
        #print('\n>> '+firstclass+'...') #class
        df.at[i, 'number_annotations'] = mixers_l_ann_nr[i]
        classpath=os.path.join(path, firstclass) # path to folder for images of  firstclass
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir_dsets, classpath, i, dset_l, mixers_l, mixers_l_catid, mixers_l_bbox, mixers_l_dset,
              mixers_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

In [72]:
#IMPORT EXCEL MANUAL EDIT #WARNING: CHECK EXCEL FIRST (#NOTE: donte use classes with missing annotations and images)
df=pd.read_excel(excelpath)
classtomix_df=df.loc[:,'classtomix_l']
classtomix_df_catid=df.loc[:,'classtomix_l_catid']
new_cat_l=copy.deepcopy(cat_l)
new_cat_l_id=copy.deepcopy(cat_l_id)
display(df)

# PARSE COLUMNS TO FIX
classtomix_l=classtomix_df.tolist()
classtomix_l_catid=classtomix_df_catid.tolist()
#convert strings to lists
for icl, cl in enumerate(classtomix_l): 
    if isinstance(classtomix_l[icl], str): classtomix_l[icl]=convert_unicode(classtomix_l[icl], method='liststr')
    if isinstance(classtomix_l_catid[icl], str): classtomix_l_catid[icl]=convert_unicode(classtomix_l_catid[icl], method='listnum')


#parse the columns(classtomix_l, classtomix_l_catid) based on the rules
#0. if both columns are empty
if classtomix_df.isnull().all() or classtomix_df_catid.isnull().all():
    raise RuntimeError("Go Back to the excel and add something to classtomix_l and classtomix_l_catid")
else:
    for ic, classes in enumerate(classtomix_df):
        #1. if only classtomix_l_catid empty - get 
        if not pd.isnull(classtomix_df.iloc[ic]) and pd.isnull(classtomix_df_catid.iloc[ic]):
            if classtomix_df.iloc[ic] == 'group_remain_classes': # group the remaining ids 
                chosed_id=np.array([x for x in classtomix_l_catid if str(x) != 'nan' and str(x) !='[]'][0])
                remain_id=np.setdiff1d(np.array(new_cat_l_id), chosed_id)
                classtomix_l_catid[ic]=remain_id.tolist()
                classtomix_l[ic]="non_aggressive" #use this to give another name 
                break #NOTE : break because this shoul be the last entry
            if classtomix_df.iloc[ic] == 'add_remain_classes': # add the remaining 
                del classtomix_l[ic] #remove "add_remain_classes" from classes list
                chosed_id=np.array([x for x in classtomix_l_catid if str(x) != 'nan' and str(x) !='[]'][0])
                remain_id=np.setdiff1d(np.array(new_cat_l_id), chosed_id)
                remain_id_l=remain_id.tolist()
                remain_cat_l=[new_cat_l[new_cat_l_id.index(id)] for id in remain_id_l] #id to names
                for remain_id in remain_id_l: classtomix_l_catid.append([remain_id])
                for remain_cat in remain_cat_l: classtomix_l.append([remain_cat] )
                break #NOTE : break because this shoul be the last entry 
        #2. if only classtomix_l empty - raise
        if pd.isnull(classtomix_df.iloc[ic]) and not pd.isnull(classtomix_df_catid.iloc[ic]): 
            raise RuntimeError('only classtomix_l empty')
        #3. if classtomix_l and classtomix_l_catid not empty - mantain
        if pd.isnull(classtomix_df.iloc[ic]) and pd.isnull(classtomix_df_catid.iloc[ic]):
            if classtomix_df.iloc[ic] == 'remove': # remove ids (needs to be the last row)
                removeidx_l=[idx for idx,cid in enumerate(classtomix_l_catid) if cid in classtomix_df_catid.iloc[ic]]
                for ridx in removeidx_l: 
                    del classtomix_l_catid[ridx]
                    del classtomix_l[ridx]

        

,mixers_l,mixers_l_catid,mixers_l_catdset,classtomix_l,classtomix_l_catid,rules,annotations_missing,images_missing,Unnamed: 8,aggressive_keywords,class,notes
0,person,[1],['COCO'],knife,"[44, 77, 125, 403, 494, 858, 1017]",To fix classes: 1) You need to fill the column...,[0],[0],NaN,"knife,sword,dagger, scissors","Knife(44), Scissors(77),butcher_knife(125), sw...","razer_blade(620),"
1,bicycle,[2],['COCO'],weapon,"[227, 328, 349, 604, 683, 742, 827, 932]",NaN,[0],[0],NaN,"weapon, gun, pistol, rifle","Gun(227), pistol(328), Rifle(349), toy_gun(604...",NaN
2,car,[3],['COCO'],bat,"[35, 92, 152, 193, 229, 243, 332, 377, 440, 45...",NaN,[0],[0],NaN,"bat,stick,pole,hammer","baseball bat(35), baseball_bat(92), chopstick(...",NaN
3,motorcycle,[4],['COCO'],NaN,NaN,NaN,[0],[0],NaN,NaN,NaN,NaN
4,airplane,[5],['COCO'],NaN,NaN,NaN,[0],[0],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1293,jewel_others,[1294],['LBO'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1294,cellphone_charger_with_cable,[1295],['LBO'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1295,little_bottle,[1296],['LBO'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1296,usb_charger,[1297],['LBO'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
print('>> Make sure everything is correct: \n1.Drop NaN if exist, but make sure the index is the same for the two! \n2.Put classtomix_l and classtomix_l_catid as a list of lists\n3. Change name of classes if you want\n')
fixempty=True
if fixempty:
    classtomix_l=[x for x in classtomix_l if str(x) != 'nan' and str(x) !='[]']
    classtomix_l_catid=[x for x in classtomix_l_catid if str(x) != 'nan' and str(x) !='[]']
#classtomix_l[1]=['non_aggressive'] #uncomment and change name
print(len(classtomix_l))
print(len(classtomix_l_catid))
print(classtomix_l[-1])
print(classtomix_l_catid[-1])

>> Make sure everything is correct: 
1.Drop NaN if exist, but make sure the index is the same for the two! 
2.Put classtomix_l and classtomix_l_catid as a list of lists
3. Change name of classes if you want

3
3
['bat']
[35, 92, 152, 193, 229, 243, 332, 377, 440, 459, 535, 994, 998]


## 4. Mix classes

In [74]:
# slow # newjson=copy.deepcopy(molajson) #do deepcopy to compare
# fast
newjson={'categories':[],'annotations':[] }
newjson['categories']=copy.copy(molajson['categories'])
newjson['annotations']=copy.copy(molajson['annotations'])

In [75]:
classtomix_l_catidx=[[cat_l_id.index(id) for id in id_l] for id_l in classtomix_l_catid]
#print(classtomix_l_catidx) # they should be less one, becacuse it is ordered
print(len(classtomix_l_catidx))

3


#### Change molajson['categories']: [{name: , id: }]  
=>  1. use first index cat id; 2. change name and change id;  remove the other categories (!!!Without ordering again the category id!!!)

In [76]:
# CHANGE NAME  & GET REMOVE List
keepidx_l=[]
keepid_l=[]
firstidx=0 # get first category id
for i,id_l in enumerate(tqdm(classtomix_l_catid)): #for each classtomix
    firstcatid=id_l[firstidx] # #category id 
    firstcatidx=classtomix_l_catidx[i][firstidx]# get cat index of first catid
    if isinstance(classtomix_l[i], list): newjson['categories'][firstcatidx]['name']=classtomix_l[i][firstidx] #change name of first id 
    else: newjson['categories'][firstcatidx]['name']=classtomix_l[i]
    assert newjson['categories'][firstcatidx]['id']==firstcatid #assert id - it should be the same
    keepidx_l.append(firstcatidx) #catidx to keep
    keepid_l.append(firstcatid) #catid to keep
keepidx_l=list(dict.fromkeys(keepidx_l)) # remove duplicates in the keep list
allidx_l=[index for index, value in enumerate(molajson['categories'])] # allidx in categories
removeidx_l=[idx for idx in allidx_l if idx not in keepidx_l] # remove idx 
removeitem_l=[newjson['categories'][removeidx] for removeidx in removeidx_l] #remove items #WARNING NECESSARY BECAUSE THE INDEX WILL CHANGE
print(len(allidx_l))
print(len(removeidx_l))
print(len(allidx_l)-len(removeidx_l))
print(keepidx_l in removeidx_l)

100%|██████████| 3/3 [00:00<00:00, 36472.21it/s]

1298
1295
3
False


REMOVE CLASSES

In [77]:
# REMOVE - newjson will be changed
for removeitem in removeitem_l: newjson['categories'].remove(removeitem)

In [78]:
print(newjson['categories'][-1])
print(molajson['categories'][-1])

{'frequency': 'c', 'id': 227, 'synset': 'gun.n.01', 'image_count': 0, 'instance_count': 0, 'synonyms': ['gun'], 'def': 'a weapon that discharges a bullet at high velocity from a metal tube', 'name': 'weapon', 'dataset': 2}
{'id': 1298, 'name': 'console_controller', 'supercategory': 'Electronic Device', 'dataset': 3}


REORDER IDs

In [79]:
# GET NEW IDs - REORDER IDs - #WARNING after remove
ct_l_id=[]
for i,c in enumerate(tqdm(newjson['categories'])):
    ct_l_id.append(c['id'])
newidx_l=[ct_l_id.index(id) for id in keepid_l] # make sure same sequence of keepid_l #SAME ORDER AS EXCEL
newid_l=[i+1 for i in range(len(keepid_l))] #reorder keepid_l
print(keepid_l)
print(newidx_l)
print(newid_l)

100%|██████████| 3/3 [00:00<00:00, 7621.39it/s]

[44, 227, 35]
[1, 2, 0]
[1, 2, 3]


In [80]:
# SORT IDS - Reorder based on Excel order - newjson will be changed
categories_l=copy.copy(newjson['categories'])
for i,idx in enumerate(newidx_l):
    categories_l[idx]['id']=newid_l[i]
for i,idx in enumerate(newidx_l):
    newjson['categories'][i]=categories_l[idx] #TODO sort the id in the correct sequence

In [81]:
#TEST
print(len(categories_l))
print(len(newjson['categories']))
print(len(molajson['categories']))
print(newjson['categories'][-1])
print(molajson['categories'][-1])

3
3
1298
{'supercategory': 'sports', 'id': 3, 'name': 'bat', 'dataset': 1}
{'id': 1298, 'name': 'console_controller', 'supercategory': 'Electronic Device', 'dataset': 3}


### QUESTION: REMOVE HYPERPARAMETERS? Mantain only id and NAME? OR irrelevant?

#### change molajson['annotations']: [{category_id: , }] 
=> 1.get annotation idx from catid; 2.update annotations id ; 3. update newjson['annotations']

In [82]:
# 1.get annotation idx from classtomix_l_catid
ann_catid_np=np.array(ann_catid)
classtomix_l_ann_catidx=[[np.where(ann_catid_np==id)[0].tolist()  for id in id_l] for id_l in classtomix_l_catid]
print(classtomix_l)
print(classtomix_l_catid)
print(keepid_l)
print(newid_l)
print(len(classtomix_l_ann_catidx[0]))

[['knife'], ['weapon'], ['bat']]
[[44, 77, 125, 403, 494, 858, 1017], [227, 328, 349, 604, 683, 742, 827, 932], [35, 92, 152, 193, 229, 243, 332, 377, 440, 459, 535, 994, 998]]
[44, 227, 35]
[1, 2, 3]
7


In [83]:
#2.update annotations ids & 3. update newjson['annotations'] with only the annotations frow classtomix_l_catid
newjson['annotations']=copy.copy(molajson['annotations']) #reset annotations
copy_ann_l=copy.copy(newjson['annotations'])
newjson['annotations']=[] #clear
for i, ann_catidx_l in enumerate(classtomix_l_ann_catidx): #only append annotations that 
    for catidx_l in ann_catidx_l:
        for catidx in catidx_l:
            copy_ann_l[catidx]['category_id']=newid_l[i] # update catid
            newjson['annotations'].append(copy_ann_l[catidx]) #update newjson with only the  (annotations sequence id will be lost
print(len(molajson['annotations']))
print(len(newjson['annotations']))

943300
10486


In [84]:
#TEST
print(molajson['annotations'][-1])
print(newjson['annotations'][-1])

{'id': 1363201, 'image_id': 194206, 'category_id': 1289, 'area': 504.0, 'bbox': [1090, 1104, 24, 21], 'iscrowd': 0, 'segmentation': [[1090, 1104, 1090, 1125, 1114, 1125, 1114, 1104]], 'dataset': 3}
{'id': 1346032, 'image_id': 183117, 'category_id': 3, 'area': 85762.0, 'bbox': [441, 839, 274, 313], 'iscrowd': 0, 'segmentation': [[441, 839, 441, 1152, 715, 1152, 715, 839]], 'dataset': 3}


### 4. Save mixed json

In [85]:
# fast
molajson['categories']=copy.copy(newjson['categories'])
molajson['annotations']=copy.copy(newjson['annotations'])

In [86]:
# save
print('\n >> SAVING...')
jsonfile=rdir+'annotations/'+injsonfile+'_mix.json'
with open(jsonfile, 'w') as f:
    json.dump(molajson, f)
print("JSON SAVED : {} \n".format(jsonfile))
for k in molajson:
    print(k, len(molajson[k]))
cat_l=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
print(cat_l)


 >> SAVING...
JSON SAVED : /mnt/Data/Work/EASYRIDE/P19/NC/yolov5/JSONS/annotations/splitann_cocotaolbo_fix_equal_reorder_cleanclass_cleanimg/train_mix.json 

info 5
licenses 9
categories 3
videos 1488
images 194943
tracks 8132
segment_info 0
annotations 10486
datasets 3
['knife', 'weapon', 'bat']


### 5. TEST MIX ANNOTATIONS DUPLICATES

In [95]:
molajson = json.load(open(rdir+'annotations/split_mola_fix_equal_reorder_nomissings/mix_aggressive_addremainclasses/test.json'))

FileNotFoundError: [Errno 2] No such file or directory: 'D:/external_datasets/MOLA/annotations/split_mola_fix_equal_reorder_nomissings/mix_aggressive_addremainclasses/test.json'

In [ ]:
for k in molajson:
    print(k, len(molajson[k]))

In [ ]:
# annotations category_id
ann_ids=[]
for an in tqdm(molajson['annotations']):
    ann_ids.append(an['id'])
print(len(ann_ids))

#TEST duplicates v3 -faster
u, c = np.unique(np.array(ann_ids), return_counts=True)
duplicates_l= u[c > 1].tolist()
print(len(duplicates_l))

In [ ]:
# categories name and id
cat_l=[]
for c in molajson['categories']:
    cat_l.append(c['name'])

In [ ]:
print(len(cat_l))
print(cat_l)
